**Keller Segel slime mould aggregation model**  
Slime molds act as agents and interact with the environment made up of cAMP molecules. The concentration of cAMP is defined everywhere in space, and it changes by its own inherent dynamics (natural decay) and by the action of agents (secretion of cAMP molecules by agents). This model can be implemented as an agent based model (ABM).

In the model, agents interact with the environment in two different ways. Ome way is the secretion of cAMP by the agents, which can be implemented by letting each agent increase the cAMP concentration in a discrete cell where it is located.  
The other way is chemotaxis. We can have each agent look at a cell randomly chosen from its neighbourhood, and move there with a probability determined by the difference in cAMP concentration $(\Delta c)$ between the neighbour cell and the cell where the agent is currently located. A ***sigmoid function***  
$P(\Delta c) = \frac{e^(\frac{\Delta c}{c_0})}{1 + e^(\frac{\Delta c}{c_0})}$  
would be suitable for this purpos, where $c_0$ is a parameter that determines how sensitive this probability is to $\Delta c$.  
$P(\Delta c)$ approaches 1 with $\Delta c \rightarrow \infty$, or 0 with $\Delta c \rightarrow -\infty$.

In [1]:
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("TkAgg")
from pylab import *


Bad key text.latex.preview in file C:\Users\Omika\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\Omika\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\Users\Omika\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpeg is saved, the default quality parameter.')
You proba

In [2]:
n = 1000 #number of agents
w = 100 #number of rows/columns in spatial array

class agent():
    pass

def initialize():
    global agents, env, nextenv
    
    agents = []
    for i in range(n):
        ag = agent()
        ag.x = np.random.randint(0, w)
        ag.y = np.random.randint(0, w)
        agents = np.append(agents, ag)
        
    env = zeros([w, w])
    nextenv = zeros([w, w])

k = 1 #rate of cAMP decay
Dc = 0.001 #diffusion constant of cAMP
Dh = 0.01 #spatial resolution for cAMP simulation
Dt = 0.01 #time resolution for cAMP simulation

def update():
    global agents, env, nextenv
    
    #simulating diffusion and evaporation of cAMP
    for x in range(w):
        for y in range(w):
            C, R, L, U, D = env[x, y], env[(x+1)%w, y], env[(x-1)%w, y], env[x, (y+1)%w], env[x, (y-1)%w]
            lap = (R + L + U + D - 4*C)/(Dh**2)
            nextenv[x, y] = env[x, y] + (-k*C + Dc*lap)*Dt
    env, nextenv = nextenv, env
    
    f = 1 #rate of cAMP secretion by an agent

    #simulating secretion of cAMP by agents
    for ag in agents:
        env[ag.x, ag.y] += f*Dt
        
    #simulating chemotaxis of agents
    for ag in agents:
        newx, newy = (ag.x + np.random.randint(-1, 2)) % w, (ag.y + np.random.randint(-1, 2)) % w
        diff = (env[newx, newy] - env[ag.x, ag.y])/0.1
        if np.random.uniform(low=0, high=1, size=1) < np.exp(diff)/(1 + np.exp(diff)):
            ag.x, ag.y = newx, newy

def observe():
    global agents, env, nextenv
    cla()
    imshow(env, cmap = cm.binary, vmin = 0, vmax = 1)
    axis("image")
    x = [ag.x for ag in agents]
    y = [ag.y for ag in agents]
    plot(y, x, "b.")

In [ ]:
import pycxsimulator
pycxsimulator.GUI(stepSize = 50).start(func=[initialize, observe, update])